# Análisis de Reservas de Hotel

## Introducción
Este proyecto tiene como objetivo realizar un análisis exhaustivo de un conjunto de datos de reservas de hotel. El análisis incluirá la exploración de datos (EDA), un análisis de negocio centrado en el impacto de las cancelaciones en los ingresos, y el entrenamiento y evaluación de modelos de clasificación para predecir el estado de las reservas.

## Diccionario de Datos
Los datos utilizados en este análisis provienen de Kaggle. A continuación, se detalla la fuente y una breve descripción de las columnas:

- **Fuente de los datos**: [Hotel Reservations Classification Dataset](https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset?resource=download)

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el dataset
df = pd.read_csv('data/Hotel Reservations.csv')
```

### Información base del dataset
A continuación, se muestran las primeras 5 filas del DataFrame, la información general del DataFrame, las estadísticas descriptivas y los valores nulos por columna, según lo proporcionado en el PRD.

```python
print("Primeras 5 filas del DataFrame:")
print(df.head().to_string())
print("\nInformación del DataFrame:")
df.info()
print("\nEstadísticas descriptivas:")
print(df.describe().to_string())
print("\nValores nulos por columna:")
print(df.isnull().sum().to_string())
```

## EDA (Análisis Exploratorio de Datos)

### Variables Numéricas
En esta sección se realizará un análisis de las variables numéricas, incluyendo distribuciones, correlaciones y detección de valores atípicos.

```python
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
print(f"Variables numéricas: {numeric_cols}")

plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_cols):
    plt.subplot(4, 4, i + 1)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribución de {col}')
    plt.tight_layout()
plt.show()

plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_cols):
    plt.subplot(4, 4, i + 1)
    sns.boxplot(y=df[col])
    plt.title(f'Boxplot de {col}')
    plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 10))
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación de Variables Numéricas')
plt.show()
```

### Variables Categóricas
Aquí se explorarán las variables categóricas, analizando la frecuencia de sus valores, su relación con la variable objetivo y posibles codificaciones.

```python
categorical_cols = df.select_dtypes(include='object').columns.tolist()
print(f"Variables categóricas: {categorical_cols}")

plt.figure(figsize=(15, 10))
for i, col in enumerate(categorical_cols):
    plt.subplot(2, 3, i + 1)
    sns.countplot(data=df, x=col, hue='booking_status')
    plt.title(f'Frecuencia de {col} por estado de reserva')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
plt.show()
```

### Variables Temporales
Se analizarán las variables relacionadas con el tiempo, como `arrival_year`, `arrival_month` y `arrival_date`, para identificar patrones estacionales o tendencias a lo largo del tiempo.

```python
# Crear una columna de fecha combinada
df['arrival_date_full'] = pd.to_datetime(df['arrival_year'].astype(str) + '-' + 
                                         df['arrival_month'].astype(str) + '-' + 
                                         df['arrival_date'].astype(str), errors='coerce')

# Eliminar filas con fechas inválidas si las hubiera
df.dropna(subset=['arrival_date_full'], inplace=True)

print("Información del DataFrame con la nueva columna de fecha:")
df.info()

# Análisis de tendencias temporales de reservas
df['month_year'] = df['arrival_date_full'].dt.to_period('M')
monthly_bookings = df.groupby('month_year').size().reset_index(name='count')
monthly_bookings['month_year'] = monthly_bookings['month_year'].astype(str)

plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_bookings, x='month_year', y='count')
plt.title('Número de Reservas por Mes/Año')
plt.xlabel('Mes/Año')
plt.ylabel('Número de Reservas')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
```

## Análisis de Negocio
En esta sección se analizará cómo las cancelaciones afectan los ingresos del hotel, así como las tendencias temporales y la relación entre otras variables y las cancelaciones.

- **Análisis de tendencias temporales**: Se examinará cómo las cancelaciones y otros factores clave varían a lo largo del tiempo.

```python
cancellations_over_time = df.groupby('month_year')['booking_status'].value_counts(normalize=True).unstack().fillna(0)
cancellations_over_time['Canceled_Rate'] = cancellations_over_time.get('Canceled', 0)

plt.figure(figsize=(12, 6))
sns.lineplot(data=cancellations_over_time, x=cancellations_over_time.index, y='Canceled_Rate')
plt.title('Tasa de Cancelación a lo largo del Tiempo')
plt.xlabel('Mes/Año')
plt.ylabel('Tasa de Cancelación')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
```

- **Análisis de la relación entre otras variables y las cancelaciones**: Se investigará la influencia de variables como el tipo de habitación, el segmento de mercado, el número de solicitudes especiales, etc., en la probabilidad de cancelación.

```python
df['is_canceled'] = df['booking_status'].apply(lambda x: 1 if x == 'Canceled' else 0)

for col in categorical_cols:
    if col != 'booking_status': # Evitar la variable objetivo
        plt.figure(figsize=(10, 6))
        sns.barplot(data=df, x=col, y='is_canceled', ci=None)
        plt.title(f'Tasa de Cancelación por {col}')
        plt.xlabel(col)
        plt.ylabel('Tasa de Cancelación')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

for col in numeric_cols:
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x='booking_status', y=col)
    plt.title(f'{col} por Estado de Reserva')
    plt.xlabel('Estado de Reserva')
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()
```

## Entrenamiento y Evaluación de Modelos

### Limpieza de Datos y Preprocesado
Se llevarán a cabo las tareas de limpieza de datos, manejo de valores nulos (si los hubiera, aunque el PRD indica que no hay), y preprocesamiento de las variables para preparar el dataset para el entrenamiento de modelos. Esto incluirá la codificación de variables categóricas y el escalado de variables numéricas si es necesario.

```python
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Codificar la variable objetivo 'booking_status'
le = LabelEncoder()
df['booking_status_encoded'] = le.fit_transform(df['booking_status'])

# Codificar variables categóricas restantes
df_encoded = pd.get_dummies(df, columns=[col for col in categorical_cols if col != 'booking_status'], drop_first=True)

# Seleccionar características y variable objetivo
X = df_encoded.drop(['Booking_ID', 'arrival_date_full', 'month_year', 'booking_status', 'booking_status_encoded'], axis=1)
y = df_encoded['booking_status_encoded']

# Escalar variables numéricas
scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("Dimensiones de los conjuntos de entrenamiento y prueba:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")
```

### Entrenamiento de Árbol de Decisión
Se entrenará un modelo de Árbol de Decisión para predecir el estado de la reserva (`booking_status`).
- **Gráfico y Evaluación**: Se visualizará el árbol de decisión y se evaluará su rendimiento utilizando métricas apropiadas.

```python
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Entrenar el modelo de Árbol de Decisión
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Predicciones
y_pred_dt = dt_model.predict(X_test)

# Evaluación
print("\nReporte de Clasificación (Árbol de Decisión):")
print(classification_report(y_test, y_pred_dt))

print("\nMatriz de Confusión (Árbol de Decisión):")
print(confusion_matrix(y_test, y_pred_dt))

print(f"Precisión del modelo de Árbol de Decisión: {accuracy_score(y_test, y_pred_dt):.4f}")

# Visualización del Árbol de Decisión (solo una parte para evitar un gráfico demasiado grande)
plt.figure(figsize=(20, 10))
plot_tree(dt_model, filled=True, feature_names=X.columns.tolist(), class_names=le.classes_,
          max_depth=3, fontsize=10)
plt.title('Árbol de Decisión (profundidad máxima = 3)')
plt.show()
```

### Entrenamiento de Random Forest
Se entrenará un modelo de Random Forest, que es un conjunto de árboles de decisión, para mejorar la precisión de la predicción.
- **Evaluación y Métricas**: Se evaluará el rendimiento del modelo de Random Forest y se presentarán métricas clave como precisión, recall, F1-score y la matriz de confusión.

```python
from sklearn.ensemble import RandomForestClassifier

# Entrenar el modelo de Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Predicciones
y_pred_rf = rf_model.predict(X_test)

# Evaluación
print("\nReporte de Clasificación (Random Forest):")
print(classification_report(y_test, y_pred_rf))

print("\nMatriz de Confusión (Random Forest):")
print(confusion_matrix(y_test, y_pred_rf))

print(f"Precisión del modelo de Random Forest: {accuracy_score(y_test, y_pred_rf):.4f}")
```

### Entrenamiento de Regresión Logística
Se entrenará un modelo de Regresión Logística, un algoritmo de clasificación lineal, para comparar su rendimiento con los modelos basados en árboles.
- **Evaluación y Métricas**: Se evaluará el rendimiento del modelo de Regresión Logística y se presentarán las métricas de evaluación correspondientes.

```python
from sklearn.linear_model import LogisticRegression

# Entrenar el modelo de Regresión Logística
lr_model = LogisticRegression(random_state=42, solver='liblinear', n_jobs=-1)
lr_model.fit(X_train, y_train)

# Predicciones
y_pred_lr = lr_model.predict(X_test)

# Evaluación
print("\nReporte de Clasificación (Regresión Logística):")
print(classification_report(y_test, y_pred_lr))

print("\nMatriz de Confusión (Regresión Logística):")
print(confusion_matrix(y_test, y_pred_lr))

print(f"Precisión del modelo de Regresión Logística: {accuracy_score(y_test, y_pred_lr):.4f}")
```
